In [1]:
import pandas as pd
import requests
from io import StringIO
from tqdm import tqdm

NASDAQ_LISTED_URL = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
OTHER_LISTED_URL  = "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt"

def load_symdir(url: str) -> pd.DataFrame:
    txt = requests.get(url, timeout=30).text
    # 마지막 트레일러 라인 제거 (File Creation Time:)
    lines = [ln for ln in txt.splitlines() if not ln.startswith("File Creation Time:")]
    df = pd.read_csv(StringIO("\n".join(lines)), sep="|", dtype=str)
    df.columns = [c.strip() for c in df.columns]
    return df

nas = load_symdir(NASDAQ_LISTED_URL)
oth = load_symdir(OTHER_LISTED_URL)

# 티커 컬럼 표준화: nas는 Symbol, other는 ACT Symbol
nas["ticker"] = nas["Symbol"]
oth["ticker"] = oth["ACT Symbol"]

all_df = pd.concat([nas, oth], ignore_index=True, sort=False)

# ETF만 필터 + 테스트이슈 제거(권장)
etf_df = all_df[all_df["ETF"].fillna("N").eq("Y")].copy()
if "Test Issue" in etf_df.columns:
    etf_df = etf_df[etf_df["Test Issue"].fillna("N").eq("N")]

# ticker 정리
etf_df["ticker"] = (
    etf_df["ticker"]
    .astype(str)
    .str.strip()
    .str.upper()
)
etf_df = etf_df[etf_df["ticker"].ne("")].drop_duplicates(subset=["ticker"]).reset_index(drop=True)

tickers = etf_df["ticker"].tolist()

print("ETF 티커 수:", len(tickers))


ETF 티커 수: 4841


In [2]:
def sanitize_tickers(tickers):
    # pandas Series / list 모두 처리
    s = pd.Series(list(tickers))

    # 문자열화 + 결측 제거
    s = s.dropna().astype(str)

    # 앞뒤 공백 제거
    s = s.str.strip()

    # 빈 문자열 제거
    s = s[s.ne("")]

    # 'nan' 같은 문자열로 들어간 것도 제거
    s = s[~s.str.lower().isin(["nan", "none", "null"])]

    # 혹시 리스트/튜플 문자열화로 이상해진 케이스 방지(공백 포함 등)
    # 미국 주식/ETF 티커는 보통 대문자, '.', '-', '^' 등 일부 기호가 섞일 수 있음
    s = s[s.str.match(r"^[A-Za-z0-9\.\-^=]+$")]

    # 대문자 통일 + 중복 제거
    s = s.str.upper().drop_duplicates()

    return s.tolist()

tickers_clean = sanitize_tickers(tickers)

In [3]:
def etf_filter(df, start_date="2000-01-01", end_date="2002-12-31"):
    start = pd.Timestamp(start_date)
    end   = pd.Timestamp(end_date)

    # 1) 2000년(1/1~12/31) 구간에 유효값이 한 번이라도 있는 ETF
    has_df = df.loc[start:end].notna().any(axis=0)

    # 2) 최신일(데이터셋 마지막 날짜)까지 유효값이 있는 ETF
    max_date = df.index.max()
    has_latest = df.loc[max_date].notna()   # 최신일 row에서 NaN이 아니면 True

    # 최종 필터
    keep = has_df & has_latest
    return keep['Close'].values[0]


In [4]:
import yfinance as yf

ticker_list = {}

for ticker in tqdm(tickers_clean[:100]):
    try:

        df = yf.download(
            ticker,
            start="2000-01-01",
            auto_adjust=True,   # 배당/분할 반영 (백테스트에 유리)
            progress=False,
            group_by="column"
        )

        if etf_filter(df):
            ticker_list[ticker] = df
            print(f"{ticker} added to ticker_list.")


    except:
        print(f"Error downloading data for {ticker}")

 75%|███████▌  | 75/100 [00:39<00:13,  1.79it/s]

BBH added to ticker_list.


100%|██████████| 100/100 [00:52<00:00,  1.91it/s]


In [5]:
ticker_list

{'BBH': Price            Close        High         Low        Open   Volume
 Ticker             BBH         BBH         BBH         BBH      BBH
 Date                                                               
 2000-01-03   17.992445   18.408010   17.269054   18.408010  1538100
 2000-01-04   16.253227   17.607663   16.230140   17.192098  1292100
 2000-01-05   16.407146   16.591842   15.760711   16.345581  1130100
 2000-01-06   16.791924   16.992011   16.453316   16.530272   757500
 2000-01-07   18.300274   18.361838   17.084360   17.238273  1663800
 ...                ...         ...         ...         ...      ...
 2025-12-15  191.110001  191.190002  190.020004  190.940002     5200
 2025-12-16  189.960007  190.940002  189.410004  190.940002     3500
 2025-12-17  189.889999  191.039993  189.889999  191.039993     5400
 2025-12-18  187.759995  189.110001  187.490005  189.110001     5000
 2025-12-19  191.869995  192.529999  189.000000  189.000000     3400
 
 [6532 rows x 5 columns]}

In [8]:
import pickle
from pathlib import Path

def save_df_dict_pickle(df_dict: dict, filepath: str):
    path = Path(filepath)
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("wb") as f:
        pickle.dump(df_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_df_dict_pickle(filepath: str) -> dict:
    path = Path(filepath)
    with path.open("rb") as f:
        return pickle.load(f)

# 사용 예시
save_df_dict_pickle(ticker_list, "./etf_prices.pkl")
ticker_list2 = load_df_dict_pickle("./etf_prices.pkl")
ticker_list2['BBH']


Price,Close,High,Low,Open,Volume
Ticker,BBH,BBH,BBH,BBH,BBH
Date,,,,,
2000-01-03,17.992445,18.408010,17.269054,18.408010,1538100
2000-01-04,16.253227,17.607663,16.230140,17.192098,1292100
2000-01-05,16.407146,16.591842,15.760711,16.345581,1130100
2000-01-06,16.791924,16.992011,16.453316,16.530272,757500
2000-01-07,18.300274,18.361838,17.084360,17.238273,1663800
...,...,...,...,...,...
2025-12-15,191.110001,191.190002,190.020004,190.940002,5200
2025-12-16,189.960007,190.940002,189.410004,190.940002,3500
